In [1]:
import pandas as pd
import numpy as np
import os, glob
import xgboost as xgb
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

/Users/lianzou/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/Users/lianzou/miniconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
filename = os.path.join(os.getcwd(), "FEATURE_ENGINEERING.txt")
df = pd.read_table(filename, low_memory=False)

In [3]:
df.drop(columns=df.columns[0], axis=1, inplace=True)
df

,ac_id,visited_page,diff_dates,Max Date Diff,Min Date Diff,Purchase 30_x,Purchase 60_x,Purchase 90_x,Purchase 120_x,Purchase 180_x,...,arr_ind_21,arr_ind_22,arr_ind_23,arr_ind_24,arr_ind_25,arr_ind_26,arr_ind_27,arr_ind_28,arr_ind_29,arr_ind_30
0,ACe761e40e-3259-4e4f-93f9-8f2f2ed34388,REWARDS,52 days,380 days,31 days,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,AC612ca133-52a6-456d-a978-e6ecfa9e87d6,PRICINGENGINE,155 days,386 days,91 days,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,AC200056f5-32de-4cbd-927d-278f3ee18282,PRICINGENGINE,257 days,336 days,5 days,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,AC200056f5-32de-4cbd-927d-278f3ee18282,PRICINGENGINE,209 days,336 days,5 days,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,AC4c2519a1-4934-47e6-8c22-2ccfa240b586,MYACCOUNT,246 days,379 days,7 days,0.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498663,ACdc6643c3-55a3-4f1b-b07a-fd486c85fb2f,PRICINGENGINE,38 days,327 days,3 days,0.0,0.0,0.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
498664,AC8d97243d-d43d-4f68-bf7f-30d07c485a54,REWARDS,88 days,224 days,5 days,0.0,0.0,0.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
498665,AC762d9492-ad39-4d14-888a-8ac279982d85,PRICINGENGINE,223 days,349 days,47 days,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
498666,AC043ad7c3-807f-4f92-9c31-3161b69e1994,REWARDS,335 days,376 days,27 days,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [4]:
feature_list = list(df.iloc[:, 0:38].select_dtypes(include="float64"))
feature_list

['Purchase 30_x',
 'Purchase 60_x',
 'Purchase 90_x',
 'Purchase 120_x',
 'Purchase 180_x',
 'Purchase 360_x',
 'Web 30_x',
 'Web 60_x',
 'Web 90_x',
 'Web 120_x',
 'Web 180_x',
 'Web 360_x',
 'Purchase 30_y',
 'Purchase 60_y',
 'Purchase 90_y',
 'Purchase 120_y',
 'Purchase 180_y',
 'Purchase 360_y',
 'Web 30_y',
 'Web 60_y',
 'Web 90_y',
 'Web 120_y',
 'Web 180_y',
 'Web 360_y',
 'new_account_indicator',
 'spend_active',
 'is_active_balance',
 'has_credit_limit_reached',
 'is_revolving',
 'has_spend_at_any_merchant',
 'arr_ind_1',
 'arr_ind_2']

In [5]:
y = df.loc[:,"arr_ind_3"]
X = df.loc[:,feature_list]

print("Number of examples: " + str(X.shape[0]))
print("\nNumber of Features:" + str(X.shape[1]))
print(str(list(X.columns)))

Number of examples: 498668

Number of Features:32
['Purchase 30_x', 'Purchase 60_x', 'Purchase 90_x', 'Purchase 120_x', 'Purchase 180_x', 'Purchase 360_x', 'Web 30_x', 'Web 60_x', 'Web 90_x', 'Web 120_x', 'Web 180_x', 'Web 360_x', 'Purchase 30_y', 'Purchase 60_y', 'Purchase 90_y', 'Purchase 120_y', 'Purchase 180_y', 'Purchase 360_y', 'Web 30_y', 'Web 60_y', 'Web 90_y', 'Web 120_y', 'Web 180_y', 'Web 360_y', 'new_account_indicator', 'spend_active', 'is_active_balance', 'has_credit_limit_reached', 'is_revolving', 'has_spend_at_any_merchant', 'arr_ind_1', 'arr_ind_2']


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1234)
print(X_train.shape)
print(X_test.shape)

(334107, 32)
(164561, 32)


In [7]:
regressor = xgb.XGBRegressor(
    n_estimators=100,
    reg_lambda=1,
    gamma=0,
    max_depth=3
)

In [8]:
regressor.fit(X_train, y_train)

/Users/lianzou/miniconda3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=3, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [9]:
pd.DataFrame(regressor.feature_importances_.reshape(1, -1), columns=feature_list)

,Purchase 30_x,Purchase 60_x,Purchase 90_x,Purchase 120_x,Purchase 180_x,Purchase 360_x,Web 30_x,Web 60_x,Web 90_x,Web 120_x,...,Web 180_y,Web 360_y,new_account_indicator,spend_active,is_active_balance,has_credit_limit_reached,is_revolving,has_spend_at_any_merchant,arr_ind_1,arr_ind_2
0,0.014046,0.019884,0.044308,0.034559,0.01175,0.004896,0.0,0.048906,0.0,0.0,...,0.038684,0.040415,0.02279,0.039843,0.052569,0.040414,0.082994,0.042848,0.0,0.0


In [10]:
y_pred = regressor.predict(X_test)

In [11]:
predictions = [round(value) for value in y_pred]

In [12]:
from sklearn.metrics import accuracy_score

In [13]:
acc_3 = accuracy_score(y_test, predictions)

In [14]:
mse_3 = mean_squared_error(y_test, y_pred)

In [15]:
feature_list = list(df.iloc[:, 0:45].select_dtypes(include="float64"))
feature_list

y = df.loc[:,"arr_ind_10"]
X = df.loc[:,feature_list]

print("Number of examples: " + str(X.shape[0]))
print("\nNumber of Features:" + str(X.shape[1]))
print(str(list(X.columns)))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1234)
print(X_train.shape)
print(X_test.shape)

Number of examples: 498668

Number of Features:39
['Purchase 30_x', 'Purchase 60_x', 'Purchase 90_x', 'Purchase 120_x', 'Purchase 180_x', 'Purchase 360_x', 'Web 30_x', 'Web 60_x', 'Web 90_x', 'Web 120_x', 'Web 180_x', 'Web 360_x', 'Purchase 30_y', 'Purchase 60_y', 'Purchase 90_y', 'Purchase 120_y', 'Purchase 180_y', 'Purchase 360_y', 'Web 30_y', 'Web 60_y', 'Web 90_y', 'Web 120_y', 'Web 180_y', 'Web 360_y', 'new_account_indicator', 'spend_active', 'is_active_balance', 'has_credit_limit_reached', 'is_revolving', 'has_spend_at_any_merchant', 'arr_ind_1', 'arr_ind_2', 'arr_ind_3', 'arr_ind_4', 'arr_ind_5', 'arr_ind_6', 'arr_ind_7', 'arr_ind_8', 'arr_ind_9']
(334107, 39)
(164561, 39)


In [16]:
regressor.fit(X_train, y_train)

/Users/lianzou/miniconda3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=3, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [17]:
pd.DataFrame(regressor.feature_importances_.reshape(1, -1), columns=feature_list)

,Purchase 30_x,Purchase 60_x,Purchase 90_x,Purchase 120_x,Purchase 180_x,Purchase 360_x,Web 30_x,Web 60_x,Web 90_x,Web 120_x,...,has_spend_at_any_merchant,arr_ind_1,arr_ind_2,arr_ind_3,arr_ind_4,arr_ind_5,arr_ind_6,arr_ind_7,arr_ind_8,arr_ind_9
0,0.029215,0.008942,0.014491,0.0,0.027795,0.00317,0.0072,0.0,0.003312,0.0,...,0.037541,0.0,0.0,0.035663,0.027691,0.032827,0.035055,0.04418,0.046186,0.028082


In [18]:
y_pred = regressor.predict(X_test)

In [19]:
predictions = [round(value) for value in y_pred]

In [20]:
accuracy = accuracy_score(y_test, predictions)
acc_10 = accuracy

In [21]:
mse_10 = mean_squared_error(y_test, y_pred)
mse_10

0.24881981174832657

In [22]:
feature_list = list(df.iloc[:, 0:55].select_dtypes(include="float64"))
feature_list

y = df.loc[:,"arr_ind_20"]
X = df.loc[:,feature_list]

print("Number of examples: " + str(X.shape[0]))
print("\nNumber of Features:" + str(X.shape[1]))
print(str(list(X.columns)))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1234)
print(X_train.shape)
print(X_test.shape)

Number of examples: 498668

Number of Features:49
['Purchase 30_x', 'Purchase 60_x', 'Purchase 90_x', 'Purchase 120_x', 'Purchase 180_x', 'Purchase 360_x', 'Web 30_x', 'Web 60_x', 'Web 90_x', 'Web 120_x', 'Web 180_x', 'Web 360_x', 'Purchase 30_y', 'Purchase 60_y', 'Purchase 90_y', 'Purchase 120_y', 'Purchase 180_y', 'Purchase 360_y', 'Web 30_y', 'Web 60_y', 'Web 90_y', 'Web 120_y', 'Web 180_y', 'Web 360_y', 'new_account_indicator', 'spend_active', 'is_active_balance', 'has_credit_limit_reached', 'is_revolving', 'has_spend_at_any_merchant', 'arr_ind_1', 'arr_ind_2', 'arr_ind_3', 'arr_ind_4', 'arr_ind_5', 'arr_ind_6', 'arr_ind_7', 'arr_ind_8', 'arr_ind_9', 'arr_ind_10', 'arr_ind_11', 'arr_ind_12', 'arr_ind_13', 'arr_ind_14', 'arr_ind_15', 'arr_ind_16', 'arr_ind_17', 'arr_ind_18', 'arr_ind_19']
(334107, 49)
(164561, 49)


In [23]:
regressor.fit(X_train, y_train)

/Users/lianzou/miniconda3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=3, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [24]:
pd.DataFrame(regressor.feature_importances_.reshape(1, -1), columns=feature_list)

,Purchase 30_x,Purchase 60_x,Purchase 90_x,Purchase 120_x,Purchase 180_x,Purchase 360_x,Web 30_x,Web 60_x,Web 90_x,Web 120_x,...,arr_ind_10,arr_ind_11,arr_ind_12,arr_ind_13,arr_ind_14,arr_ind_15,arr_ind_16,arr_ind_17,arr_ind_18,arr_ind_19
0,0.0,0.0,0.013213,0.021328,0.004688,0.005501,0.007939,0.01272,0.0,0.0,...,0.033454,0.022337,0.029079,0.017054,0.042103,0.039916,0.034142,0.017215,0.023361,0.015149


In [25]:
y_pred = regressor.predict(X_test)

In [26]:
predictions = [round(value) for value in y_pred]

In [27]:
accuracy = accuracy_score(y_test, predictions)
acc_20 = accuracy

In [28]:
mse_20 = mean_squared_error(y_test, y_pred)
mse_20

0.14978061115228625

In [29]:
feature_list = list(df.iloc[:, 0:64].select_dtypes(include="float64"))
feature_list

y = df.loc[:,"arr_ind_29"]
X = df.loc[:,feature_list]

print("Number of examples: " + str(X.shape[0]))
print("\nNumber of Features:" + str(X.shape[1]))
print(str(list(X.columns)))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1234)
print(X_train.shape)
print(X_test.shape)

Number of examples: 498668

Number of Features:58
['Purchase 30_x', 'Purchase 60_x', 'Purchase 90_x', 'Purchase 120_x', 'Purchase 180_x', 'Purchase 360_x', 'Web 30_x', 'Web 60_x', 'Web 90_x', 'Web 120_x', 'Web 180_x', 'Web 360_x', 'Purchase 30_y', 'Purchase 60_y', 'Purchase 90_y', 'Purchase 120_y', 'Purchase 180_y', 'Purchase 360_y', 'Web 30_y', 'Web 60_y', 'Web 90_y', 'Web 120_y', 'Web 180_y', 'Web 360_y', 'new_account_indicator', 'spend_active', 'is_active_balance', 'has_credit_limit_reached', 'is_revolving', 'has_spend_at_any_merchant', 'arr_ind_1', 'arr_ind_2', 'arr_ind_3', 'arr_ind_4', 'arr_ind_5', 'arr_ind_6', 'arr_ind_7', 'arr_ind_8', 'arr_ind_9', 'arr_ind_10', 'arr_ind_11', 'arr_ind_12', 'arr_ind_13', 'arr_ind_14', 'arr_ind_15', 'arr_ind_16', 'arr_ind_17', 'arr_ind_18', 'arr_ind_19', 'arr_ind_20', 'arr_ind_21', 'arr_ind_22', 'arr_ind_23', 'arr_ind_24', 'arr_ind_25', 'arr_ind_26', 'arr_ind_27', 'arr_ind_28']
(334107, 58)
(164561, 58)


In [30]:
regressor.fit(X_train, y_train)

/Users/lianzou/miniconda3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=3, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [31]:
pd.DataFrame(regressor.feature_importances_.reshape(1, -1), columns=feature_list)

,Purchase 30_x,Purchase 60_x,Purchase 90_x,Purchase 120_x,Purchase 180_x,Purchase 360_x,Web 30_x,Web 60_x,Web 90_x,Web 120_x,...,arr_ind_19,arr_ind_20,arr_ind_21,arr_ind_22,arr_ind_23,arr_ind_24,arr_ind_25,arr_ind_26,arr_ind_27,arr_ind_28
0,0.00327,0.00963,0.0,0.00983,0.008138,0.024823,0.003962,0.0,0.0,0.0,...,0.021187,0.031845,0.009927,0.017771,0.011555,0.057383,0.006063,0.058106,0.009006,0.082243


In [32]:
y_pred = regressor.predict(X_test)

In [33]:
predictions = [round(value) for value in y_pred]

In [34]:
accuracy = accuracy_score(y_test, predictions)
acc_29 = accuracy

In [35]:
mse_29 = mean_squared_error(y_test, y_pred)

In [42]:
print("Mean Squared Error for Each Model")
print("Day 3: ", mse_3)
print("Day 10: ",mse_10)
print("Day 20: ",mse_20)
print("Day 29: ", mse_29)

print("\nAccuracy for Each Model")
print("Day 3: ",acc_3)
print("Day 10: ",acc_10)
print("Day 20: ",acc_20)
print("Day 29: ",acc_29)

Mean Squared Error for Each Model
Day 3:  0.07828721033940221
Day 10:  0.24881981174832657
Day 20:  0.14978061115228625
Day 29:  0.015992833291714897

Accuracy for Each Model
Day 3:  0.9141534142354507
Day 10:  0.5252094967823482
Day 20:  0.8162322786079326
Day 29:  0.983696015459313
